In [ ]:
from dreye.core import Spectrum, SpectrumMeasurement, \
    MeasuredSpectrum, convert_measurement, CalibrationSpectrum, \
    AbstractSpectrum, Signal, Domain
from dreye.core import LogPhotoreceptor, \
    RelativeOpsinSensitivity, Spectrum, Domain
from dreye.stimuli.tempochromatic.noise import PRWhiteNoiseStimulus, \
    PRBrownNoiseStimulus, TransformWhiteNoiseStimulus, \
    TransformBrownNoiseStimulus, IlluminantWhiteNoiseStimulus, \
    IlluminantBgWhiteNoiseStimulus
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from dreye.core import create_gaussian_spectrum
np.random.seed(10)

In [ ]:
wl = np.arange(300, 600)
s = norm.pdf(wl, 450, 10)
s /= np.sum(s)
noise = 0.01
s_noise = s + np.random.normal(0, noise, size=s.shape)
ops = norm.pdf(wl, 433, 40)

In [ ]:
opsin2 = RelativeOpsinSensitivity(
    norm.pdf(wl[:,None], asarray([[330, 450, 560]]), 40),
    domain=wl, 
)

In [ ]:
plt.plot(wl, opsin2)

In [ ]:
pr = LogPhotoreceptor(opsin2)
cal = CalibrationSpectrum(
    np.ones(wl.shape), 
    wl, 
    area=1
)

In [ ]:
pr.sensitivity.labels

In [ ]:
for idx, led in enumerate([300, 360, 400, 500]):
    
    s2 = norm.pdf(wl, led, 20)
    if idx % 2:
        mess = convert_measurement(
            AbstractSpectrum(
                10 * s2 + np.random.normal(0, noise, size=s.shape), 
                wl
            ), cal, 1
        ).uE
    else:
        mess = convert_measurement(
            AbstractSpectrum(
                s2 + np.random.normal(0, noise, size=s.shape), 
                wl
            ), cal, 1
        ).uE

    for i in np.arange(2, 10)[::asarray([1, -1])[idx % 2]]:
        m = convert_measurement(
            AbstractSpectrum(
                i * s2 + np.random.normal(0, noise, size=s.shape), 
                wl
            ),
            cal, 1
        ).irradiance

        mess = mess.concat(
            m, 
        )

    ms = MeasuredSpectrum(
        mess,
        labels=Domain(np.arange(1, 10), units='volts'), 
        smoothing_window=25, 
    ).to_spectrum_measurement(
        name=f'LED{led}',
        zero_boundary=asarray([0, 10])[idx % 2], 
        max_boundary=asarray([10, 0])[idx % 2]
        #zero_is_lower=not bool(idx%2)
    )
    
        
    if idx == 0:
        mss = ms
    else:
        mss = mss.concat(ms)

In [ ]:
ms.max_boundary, ms.zero_boundary

In [ ]:
ms.labels._zero_is_lower

In [ ]:
mss.zero_is_lower

In [ ]:
mss.save('test_spms.json')

In [ ]:
mss2 = mss.load('test_spms.json')

In [ ]:
mss2.label_names

In [ ]:
mss.max_boundary

In [ ]:
mss.zero_is_lower

In [ ]:
mss.label_names

In [ ]:
plt.plot(mss.domain, mss)

In [ ]:
plt.plot(mss.normalized_spectrum)

In [ ]:
from scipy.stats import norm
plt.plot(norm.pdf(np.arange(-1, 1, 0.1)))

In [ ]:
gaussians = create_gaussian_spectrum(
    wl, 
    [340, 440, 540], 
    intensity=1, 
    background=np.ones(wl.shape)/10000, 
    add_background=True, 
    cdf=False,
    filter=True,
)

In [ ]:
plt.plot(gaussians)

In [ ]:
background = Spectrum(gaussians[:, 2]/1, wl)

In [ ]:
plt.plot(background)

In [ ]:
pr.get_A(
    mss, 
    background, 
    False
)

In [ ]:
pr.get_qs(
    mss, 
    gaussians, 
    background=background, 
    units=False, 
    return_A=False
)

In [ ]:
output, res = pr.fit(
    mss, 
    gaussians, 
    background=background, 
    return_res=True
)

In [ ]:
plt.plot(output.T)

In [ ]:
wn = PRWhiteNoiseStimulus(
    pr, 
    mss, 
    background,
    rate=60, 
    n_channels=3, 
    mean=0, 
    var=asarray([0.01,0.02,0.1]), 
    minimum=-100, 
    maximum=100, 
    iterations=2, 
    start_delay=1, 
    end_dur=1, 
    pause_dur=1, 
    seed=10, 
    filter_style='window', 
    filter_function='exponential', 
    filter_along_axis=0, 
    filter_kwargs={'M':5, 'tau':2.5}
)

In [ ]:
wn.create()

In [ ]:
plt.plot(wn.metadata['channel_weights'])

In [ ]:
plt.plot(wn.metadata['target_signal'][:, 1])
plt.plot(wn.signal[:, 1])

In [ ]:
plt.plot(wn.metadata['random_signal'])

In [ ]:
plt.plot(wn.metadata['random_signal'])

In [ ]:
wn.transform()

In [ ]:
wn.stimulus

In [ ]:
plt.plot(wn.stimulus)

In [ ]:
wn = TransformWhiteNoiseStimulus(
    np.ones(3)[:, None], 
    pr, 
    mss, 
    background,
    rate=60, 
    n_channels=1, 
    mean=0, 
    var=0.1, 
    minimum=-100, 
    maximum=100, 
    iterations=2, 
    start_delay=2, 
    end_dur=2, 
    pause_dur=2, 
    seed=10, 
    filter_style='window', 
    filter_function='exponential', 
    filter_along_axis=0, 
    filter_kwargs={'M':61, 'tau':20}
)

In [ ]:
wn.create()

In [ ]:
plt.plot(wn.metadata['channel_weights'])

In [ ]:
plt.plot(wn.metadata['target_signal'], color='black')
plt.plot(wn.signal)

In [ ]:
plt.plot(wn.metadata['res_signal'])

In [ ]:
plt.plot(wn.metadata['lineartransform_target_signal'])
plt.plot(wn.metadata['lineartransform_signal'])

In [ ]:
plt.plot(wn.stimulus[:, 0])

In [ ]:
gaussians2 = create_gaussian_spectrum(
    wl, 
    [500, 300], 
    intensity=0.5, 
    background=np.ones(wl.shape), 
    add_background=True, 
    cdf=None,
    filter=True,
)

In [ ]:
plt.plot(gaussians2)

In [ ]:
wn = IlluminantWhiteNoiseStimulus(
    gaussians2/500, 
    pr, 
    mss, 
    background,
    rate=60, 
    n_channels=2, 
    mean=0, 
    var=[0.1, 0.5], 
    minimum=-100, 
    maximum=100, 
    iterations=2, 
    start_delay=2, 
    end_dur=2, 
    pause_dur=2, 
    seed=10, 
    filter_style='window', 
    filter_function='exponential', 
    filter_along_axis=0, 
    filter_kwargs={'M':61, 'tau':20}
)

In [ ]:
wn.linear_transform

In [ ]:
wn.create()

In [ ]:
plt.plot(wn.metadata['target_signal'][:, 1])
plt.plot(wn.signal[:, 1])

In [ ]:
plt.plot(wn.metadata['lineartransform_target_signal'])

In [ ]:
wn = IlluminantBgWhiteNoiseStimulus(
    gaussians2, 
    pr, 
    mss, 
    background,
    rate=60, 
    n_channels=2, 
    mean=0, 
    var=[0.1, 0.5], 
    minimum=None, 
    maximum=None, 
    iterations=2, 
    start_delay=2, 
    end_dur=2, 
    pause_dur=2, 
    seed=10, 
    filter_style='window', 
    filter_function='exponential', 
    filter_along_axis=0, 
    filter_kwargs={'M':61, 'tau':20}, 
    bg_func=lambda x, bg: (x * bg) + bg
)

In [ ]:
wn.create()

In [ ]:
plt.plot(wn.signal)
plt.plot(wn.metadata['target_signal'])

In [ ]:
plt.plot(wn.metadata['original_signal'])

In [ ]:
plt.plot(wn.metadata['res_signal'])

In [ ]:
wn.events

In [ ]:
from dreye.stimuli.tempochromatic.step import (
    PRStepStimulus, TransformStepStimulus
)

In [ ]:
step = PRStepStimulus(
    photoreceptor=pr, 
    spectrum_measurement=mss, 
    background=background,
    values={
        'rh3': [0.2, 0.4, 0.3], 
        'rh4': [0.4, 0.2, 0.1], 
        'rh5': [0.2, 0.1, 0.3]
    }, 
    separate_channels=True, 
    repetitions=3, 
    iterations=3, 
    rate=60., 
    durations=[3, 3], 
    pause_durations=[2, 3], 
    randomize=True, 
    start_delay=1, 
    end_dur=1
)

In [ ]:
step.create()

In [ ]:
plt.plot(step.timestamps, step.signal)

In [ ]:
plt.plot(step.metadata['target_signal'])

In [ ]:
plt.plot(step.metadata['res_signal'])

In [ ]:
step.events

In [ ]:
step.values

In [ ]:
step = TransformStepStimulus(
    linear_transform=asarray([
        [0.5, -0.5, 0.5], [1, 1, -2]
    ]).T,
    linear_transform_labels=['a', 'b'],
    photoreceptor=pr, 
    spectrum_measurement=mss, 
    background=background,
    values={
        'rh3': [0.2, 0.4, 0.3], 
        'rh4': [0.4, 0.2, 0.1], 
    }, 
    separate_channels=True, 
    repetitions=1, 
    iterations=1, 
    rate=60., 
    durations=[3, 3], 
    pause_durations=[2, 3], 
    randomize=True, 
    start_delay=1, 
    end_dur=1
)

In [ ]:
step.create()

In [ ]:
plt.plot(step.signal)

In [ ]:
plt.plot(step.metadata['target_signal'])

In [ ]:
plt.plot(step.metadata['res_signal'])

In [ ]:
plt.plot(step.metadata['lineartransform_signal'])

In [ ]:
plt.plot(step.metadata['lineartransform_target_signal'])

In [ ]:
step.transform()

In [ ]:
plt.plot(step.stimulus)

In [ ]:
step.events

In [ ]:
wn.events

In [ ]:
wn.save('test_wn_stim.json')

In [ ]:
step.save('test_step_stim.json')

In [ ]:
step2 = PRStepStimulus.load('test_step_stim.json')

In [ ]:
wn2 = IlluminantWhiteNoiseStimulus.load('test_wn_stim.json')

In [ ]:
step2.settings.keys()

In [ ]:
wn2.settings.keys()

In [ ]:
wn2.settings['bg_func']

In [ ]:
step2

In [ ]:
wn2